In [ ]:
import corpora
corpora.shakespeare_plays

In [ ]:
import nltk
nltk.download('punkt')

# Preprocessing

In [ ]:
import preprocessing
from __future__ import print_function
corpus = corpora.shakespeare_sonnets
chars, char2idx = preprocessing.get_chars(corpus)

# Model specification

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
from keras.models import Sequential

from keras.layers import Dense, Activation, LSTM, Reshape, TimeDistributed
from keras.callbacks import Callback


def get_model(num_timesteps, num_chars, hidden_dims, batch_size):
    model = Sequential()
    model.add(LSTM(output_dim=hidden_dims, 
                   batch_input_shape=[batch_size, num_timesteps, num_chars], 
                   return_sequences=True, 
                   stateful=True))
    model.add(LSTM(output_dim=hidden_dims, return_sequences=True, stateful=True))
    model.add(TimeDistributed(Dense(num_chars), input_shape=(num_timesteps, hidden_dims)))
    model.add(Activation("softmax"))
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

class ResetStates(Callback):

    def on_epoch_begin(self, epoch, logs={}):
        self.model.reset_states()

# Training

In [ ]:
def train_model(num_timesteps, hidden_dims, batch_size, num_epochs, corpus, char2idx, model=None):
    num_chars = len(char2idx)
    model = model if model else get_model(num_timesteps, num_chars, hidden_dims, batch_size)
    examples = preprocessing.vectorized_example_stream(corpus, num_timesteps, batch_size, char2idx, word_level=False)
    total_num_chars = preprocessing.count_chars(corpus)
    total_num_chars = total_num_chars - total_num_chars % (num_timesteps * batch_size)
    samples_per_epoch = total_num_chars//num_timesteps
    model.fit_generator(examples, samples_per_epoch, num_epochs, callbacks=[ResetStates()])
    return model

In [ ]:
num_timesteps = 50
hidden_dims = 128
batch_size = 100
num_epochs = 1
trained_model = train_model(num_timesteps, hidden_dims, batch_size, 
                            num_epochs, corpus, char2idx, model=None)

In [ ]:
model_name = 'charlevel.%s.h5' % os.path.basename(corpus)
trained_model.save_weights(model_name, overwrite=True)

# Inference

In [ ]:
import numpy as np

def generate_text(model, seed_str, temperature, char2idx, idx2chars, N):
    model.reset_states()
    # first we initialize the state of the LSTM using the seed_str
    for seed_char in seed_str:
        seed_char_idx = char2idx[seed_char]
        x = np.zeros(shape=model.input_shape)
        x[0, 0, seed_char_idx] = 1
        probs = model.predict(x, verbose=0)
        
    # now we start generating text
    probs = probs[0,0,:]
    next_char_idx = sample(probs, temperature)
    generated_text_idx = [next_char_idx]
    generated_text = [idx2chars[next_char_idx]]
    x = np.zeros(shape=model.input_shape)
    for i in xrange(N - 1):
        last_char_idx = generated_text_idx[-1]
        x = np.zeros(shape=model.input_shape)
        x[0, 0, last_char_idx] = 1
        probs = model.predict(x, verbose=0)
        probs = probs[0,0,:]
        next_char = sample(probs, temperature)
        generated_text_idx.append(next_char)
        generated_text.append(idx2chars[next_char])        
    return generated_text
    
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [ ]:
seed_str = '''123'''
model_name = 'charlevel.%s.h5' % os.path.basename(corpus)
hidden_dims = 128
trained_model_test = get_model(1, len(char2idx), hidden_dims, 1)
trained_model_test.load_weights(model_name)
generated_text = generate_text(trained_model_test, seed_str, 0.1, char2idx, chars, 5000)


In [ ]:
print(seed_str + ''.join(generated_text))